Dream Housing Finance company deals in all home loans. They have presence across all urban, semi urban and rural areas. Customer first apply for home loan after that company validates the customer eligibility for loan.

Problem
Company wants to automate the loan eligibility process (real time) based on customer detail provided while filling online application form. These details are Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History and others. To automate this process, they have given a problem to identify the customers segments, those are eligible for loan amount so that they can specifically target these customers. Here they have provided a partial data set.


Variable - Description
Loan_ID - Unique Loan ID
Gender - Male/ Female
Married - Applicant married (Y/N)
Dependents - Number of dependents
Education - Applicant Education (Graduate/ Under Graduate)
Self_Employed - Self employed (Y/N)
ApplicantIncome  - Applicant income
CoapplicantIncome - Coapplicant income
LoanAmount - Loan amount in thousands
Loan_Amount_Term - Term of loan in months
Credit_History - credit history meets guidelines
Property_Area - Urban/ Semi Urban/ Rural
Loan_Status - Loan approved (Y/N)

Importing the Libs

In [262]:
import pandas as pd
import numpy as np
import scipy as sc
train = pd.read_csv('train_loan.csv')
test = pd.read_csv('test_loan.csv')

In [265]:
len(train)

614

In [174]:
train.describe()

ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
count       614.000000         614.000000  592.000000         600.00000   
mean       5403.459283        1621.245798  146.412162         342.00000   
std        6109.041673        2926.248369   85.587325          65.12041   
min         150.000000           0.000000    9.000000          12.00000   
25%        2877.500000           0.000000  100.000000         360.00000   
50%        3812.500000        1188.500000  128.000000         360.00000   
75%        5795.000000        2297.250000  168.000000         360.00000   
max       81000.000000       41667.000000  700.000000         480.00000   

       Credit_History  
count      564.000000  
mean         0.842199  
std          0.364878  
min          0.000000  
25%          1.000000  
50%          1.000000  
75%          1.000000  
max          1.000000

In [175]:
train.head()

Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  
2             1.0         Urban           Y  
3             1.0         Urban           Y  
4             1.0         Urban           Y

In [200]:
train['Loan_Status'] = train['Loan_Status'].apply(lambda x: 0 if x=='N' else 1 )
train['Self_Employed'].fillna((train['Self_Employed'].mode()[0]), inplace=True)
train['Self_Employed'] = train['Self_Employed'].apply(lambda x: 0 if x=='No' else 1 )
train['LoanAmount'].fillna((train['LoanAmount'].mean()), inplace=True)
train['Dependents'] =  train['Dependents'].replace('3+',3)
train['Dependents'].fillna((train['Dependents'].mode()[0]), inplace=True)
train['Credit_History'].fillna((train['Credit_History'].mode()[0]), inplace=True)
train['Gender'].fillna((train['Gender'].mode()[0]), inplace=True)
train['Gender'] = train['Gender'].astype("category")
train['Loan_Amount_Term'].fillna((train['Loan_Amount_Term'].mode()[0]), inplace=True)
train['Education'] = train['Education'].astype("category")
train['Property_Area'] = train['Property_Area'].astype("category")
train=train[((train.ApplicantIncome - train.ApplicantIncome.mean()) / train.ApplicantIncome.std()).abs() < 3]
train=train[((train.CoapplicantIncome - train.CoapplicantIncome.mean()) / train.CoapplicantIncome.std()).abs() < 3]

In [201]:
train['Total_Income'] = train['ApplicantIncome'] + train['CoapplicantIncome']
train['LoanAmtPerMon'] = train['LoanAmount'] / train['Loan_Amount_Term']
train['TotIncomeByLoan'] =  train['Total_Income'] /  train['LoanAmount']

In [202]:
train.count()

Loan_ID              600
Gender               600
Married              597
Dependents           600
Education            600
Self_Employed        600
ApplicantIncome      600
CoapplicantIncome    600
LoanAmount           600
Loan_Amount_Term     600
Credit_History       600
Property_Area        600
Loan_Status          600
Total_Income         600
LoanAmtPerMon        600
TotIncomeByLoan      600
dtype: int64

In [244]:
train = pd.get_dummies(train)
train.count()

Self_Employed              600
ApplicantIncome            600
CoapplicantIncome          600
LoanAmount                 600
Loan_Amount_Term           600
                          ... 
Education_Graduate         600
Education_Not Graduate     600
Property_Area_Rural        600
Property_Area_Semiurban    600
Property_Area_Urban        600
Length: 623, dtype: int64

In [237]:
import numpy as np
msk = np.random.rand(len(train)) < 0.8

new_valid = train[~msk]
new_train = train[msk]

In [241]:
new_train.shape
new_valid.shape

(131, 623)

In [243]:
new_train.head()

Self_Employed  ApplicantIncome  CoapplicantIncome  LoanAmount  \
0              0             5849                0.0  146.412162   
1              0             4583             1508.0  128.000000   
2              1             3000                0.0   66.000000   
5              1             5417             4196.0  267.000000   
6              0             2333             1516.0   95.000000   

   Loan_Amount_Term  Credit_History  Loan_Status  Total_Income  LoanAmtPerMon  \
0             360.0             1.0            1        5849.0       0.406700   
1             360.0             1.0            0        6091.0       0.355556   
2             360.0             1.0            1        3000.0       0.183333   
5             360.0             1.0            1        9613.0       0.741667   
6             360.0             1.0            1        3849.0       0.263889   

   TotIncomeByLoan  ...  Married_Yes  Dependents_3  Dependents_0  \
0        39.948867  ...            0             0             1   
1        47.585938  ...            1             0             0   
2        45.454545  ...            1             0             1   
5        36.003745  ...            1             0             0   
6        40.515789  ...            1             0             1   

   Dependents_1  Dependents_2  Education_Graduate  Education_Not Graduate  \
0             0             0                   1                       0   
1             1             0                   1                       0   
2             0             0                   1                       0   
5             0             1                   1                       0   
6             0             0                   0                       1   

   Property_Area_Rural  Property_Area_Semiurban  Property_Area_Urban  
0                    0                        0                    1  
1                    1                        0                    0  
2                    0                        0                    1  
5                    0                        0                    1  
6                    0                        0                    1  

[5 rows x 623 columns]

In [267]:
new_valid_loan_Status = new_valid['Loan_Status']
new_valid.drop(['Loan_Status'],axis=1)             
new_train_loan_Status = new_train['Loan_Status']
new_train.drop(['Loan_Status'],axis=1)

Self_Employed  ApplicantIncome  CoapplicantIncome  LoanAmount  \
0                0             5849                0.0  146.412162   
1                0             4583             1508.0  128.000000   
2                1             3000                0.0   66.000000   
5                1             5417             4196.0  267.000000   
6                0             2333             1516.0   95.000000   
..             ...              ...                ...         ...   
608              0             3232             1950.0  108.000000   
609              0             2900                0.0   71.000000   
610              0             4106                0.0   40.000000   
611              0             8072              240.0  253.000000   
613              1             4583                0.0  133.000000   

     Loan_Amount_Term  Credit_History  Total_Income  LoanAmtPerMon  \
0               360.0             1.0        5849.0       0.406700   
1               360.0             1.0        6091.0       0.355556   
2               360.0             1.0        3000.0       0.183333   
5               360.0             1.0        9613.0       0.741667   
6               360.0             1.0        3849.0       0.263889   
..                ...             ...           ...            ...   
608             360.0             1.0        5182.0       0.300000   
609             360.0             1.0        2900.0       0.197222   
610             180.0             1.0        4106.0       0.222222   
611             360.0             1.0        8312.0       0.702778   
613             360.0             0.0        4583.0       0.369444   

     TotIncomeByLoan  Loan_ID_LP001002  ...  Married_Yes  Dependents_3  \
0          39.948867                 1  ...            0             0   
1          47.585938                 0  ...            1             0   
2          45.454545                 0  ...            1             0   
5          36.003745                 0  ...            1             0   
6          40.515789                 0  ...            1             0   
..               ...               ...  ...          ...           ...   
608        47.981481                 0  ...            1             0   
609        40.845070                 0  ...            0             0   
610       102.650000                 0  ...            1             1   
611        32.853755                 0  ...            1             0   
613        34.458647                 0  ...            0             0   

     Dependents_0  Dependents_1  Dependents_2  Education_Graduate  \
0               1             0             0                   1   
1               0             1             0                   1   
2               1             0             0                   1   
5               0             0             1                   1   
6               1             0             0                   0   
..            ...           ...           ...                 ...   
608             1             0             0                   1   
609             1             0             0                   1   
610             0             0             0                   1   
611             0             1             0                   1   
613             1             0             0                   1   

     Education_Not Graduate  Property_Area_Rural  Property_Area_Semiurban  \
0                         0                    0                        0   
1                         0                    1                        0   
2                         0                    0                        0   
5                         0                    0                        0   
6                         1                    0                        0   
..                      ...                  ...                      ...   
608                       0                    1                  

In [249]:
from sklearn.linear_model import LogisticRegression
import sklearn
logistic_clf = LogisticRegression(random_state=0).fit(new_train,new_train_loan_Status)

C:\Akshay\Train\Applications\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [251]:
pred_logistic = logistic_clf.predict(new_valid)

In [253]:
pred_logistic

array([1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1],
      dtype=int64)

In [255]:
accuracy_logistic = sklearn.metrics.accuracy_score(new_valid_loan_Status, pred_logistic)
accuracy_logistic

1.0

In [258]:
from sklearn.ensemble import RandomForestClassifier
Random_forest = RandomForestClassifier(n_estimators=100, random_state = 42).fit(new_train,new_train_loan_Status)

In [260]:
pred_Random_forest = Random_forest.predict(new_valid)
pred_Random_forest

array([1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1],
      dtype=int64)

In [298]:
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.ensemble import RandomForestClassifier
test = pd.read_csv('test_loan.csv')

AttributeError: 'DataFrame' object has no attribute 'record'

In [311]:
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.ensemble import RandomForestClassifier
train = pd.read_csv('train_loan.csv')
test = pd.read_csv('test_loan.csv')
train['Loan_Status'] = train['Loan_Status'].apply(lambda x: 0 if x=='N' else 1 )
train['Self_Employed'].fillna((train['Self_Employed'].mode()[0]), inplace=True)
train['Self_Employed'] = train['Self_Employed'].apply(lambda x: 0 if x=='No' else 1 )
train['LoanAmount'].fillna((train['LoanAmount'].mean()), inplace=True)
train['Dependents'] =  train['Dependents'].replace('3+',3)
train['Dependents'].fillna((train['Dependents'].mode()[0]), inplace=True)
train['Credit_History'].fillna((train['Credit_History'].mode()[0]), inplace=True)
train['Gender'].fillna((train['Gender'].mode()[0]), inplace=True)
train['Gender'] = train['Gender'].astype("category")
train['Loan_Amount_Term'].fillna((train['Loan_Amount_Term'].mode()[0]), inplace=True)
train['Education'] = train['Education'].astype("category")
train['Property_Area'] = train['Property_Area'].astype("category")
train=train[((train.ApplicantIncome - train.ApplicantIncome.mean()) / train.ApplicantIncome.std()).abs() < 3]
train=train[((train.CoapplicantIncome - train.CoapplicantIncome.mean()) / train.CoapplicantIncome.std()).abs() < 3]
train['Total_Income'] = train['ApplicantIncome'] + train['CoapplicantIncome']
train['LoanAmtPerMon'] = train['LoanAmount'] / train['Loan_Amount_Term']
train['TotIncomeByLoan'] =  train['Total_Income'] /  train['LoanAmount']
new_train=train
new_train_loan_Status = new_train['Loan_Status']
new_train=new_train.drop(['Loan_Status'],axis=1)
new_train=new_train.drop(['Loan_ID'],axis=1)


#Test
test['Self_Employed'].fillna((test['Self_Employed'].mode()[0]), inplace=True)
test['Self_Employed'] = test['Self_Employed'].apply(lambda x: 0 if x=='No' else 1 )
test['LoanAmount'].fillna((test['LoanAmount'].mean()), inplace=True)
test['Dependents'] =  test['Dependents'].replace('3+',3)
test['Dependents'].fillna((test['Dependents'].mode()[0]), inplace=True)
test['Credit_History'].fillna((test['Credit_History'].mode()[0]), inplace=True)
test['Gender'].fillna((test['Gender'].mode()[0]), inplace=True)
test['Gender'] = test['Gender'].astype("category")
test['Loan_Amount_Term'].fillna((test['Loan_Amount_Term'].mode()[0]), inplace=True)
test['Education'] = test['Education'].astype("category")
test['Property_Area'] = test['Property_Area'].astype("category")
test['Total_Income'] = test['ApplicantIncome'] + test['CoapplicantIncome']
test['LoanAmtPerMon'] = test['LoanAmount'] / test['Loan_Amount_Term']
test['TotIncomeByLoan'] =  test['Total_Income'] /  test['LoanAmount']
new_test=test.drop(['Loan_ID'],axis=1)
#Mark New Train with Training Column and Test with Test Column. If we use get_dummies on different data set
#Its possible that data set cannot match due to categorical values
new_train['Label'] = 1
new_test['Label'] = 0
# Concat
concat_df = pd.concat([new_train , test])
# Create your dummies
features_df = pd.get_dummies(concat_df)
# Split your data
train_df = features_df[features_df['Label'] == 1]
test_df = features_df[features_df['Label'] == 0]
#Running Random Forest
#Random_forest = RandomForestClassifier(n_estimators=100, random_state = 42).fit(train_df,new_train_loan_Status)
#pred_Random_forest = Random_forest.predict(test_df)
#pred_Random_forest
#final_test=test
#final_test['Loan_Status']=pred_Random_forest
#final_test
new_train

C:\Akshay\Train\Applications\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Gender Married Dependents     Education  Self_Employed  ApplicantIncome  \
0      Male      No          0      Graduate              0             5849   
1      Male     Yes          1      Graduate              0             4583   
2      Male     Yes          0      Graduate              1             3000   
3      Male     Yes          0  Not Graduate              0             2583   
4      Male      No          0      Graduate              0             6000   
5      Male     Yes          2      Graduate              1             5417   
6      Male     Yes          0  Not Graduate              0             2333   
7      Male     Yes          3      Graduate              0             3036   
8      Male     Yes          2      Graduate              0             4006   
10     Male     Yes          2      Graduate              0             3200   
11     Male     Yes          2      Graduate              0             2500   
12     Male     Yes          2      Graduate              0             3073   
13     Male      No          0      Graduate              0             1853   
14     Male     Yes          2      Graduate              0             1299   
15     Male      No          0      Graduate              0             4950   
16     Male      No          1  Not Graduate              0             3596   
17   Female      No          0      Graduate              0             3510   
18     Male     Yes          0  Not Graduate              0             4887   
19     Male     Yes          0      Graduate              0             2600   
20     Male     Yes          0  Not Graduate              0             7660   
21     Male     Yes          1      Graduate              0             5955   
22     Male     Yes          0  Not Graduate              0             2600   
23     Male     Yes          2  Not Graduate              0             3365   
24     Male     Yes          1      Graduate              0             3717   
25     Male     Yes          0      Graduate              1             9560   
26     Male     Yes          0      Graduate              0             2799   
27     Male     Yes          2  Not Graduate              0             4226   
28     Male      No          0  Not Graduate              0             1442   
29   Female      No          2      Graduate              0             3750   
30     Male     Yes          1      Graduate              0             4166   
31     Male      No          0      Graduate              0             3167   
32     Male      No          1      Graduate              1             4692   
33     Male     Yes          0      Graduate              0             3500   
34     Male      No          3      Graduate              0            12500   
35     Male     Yes          0      Graduate              0             2275   
36     Male     Yes          0      Graduate              0             1828   
37   Female     Yes          0      Graduate              0             3667   
38     Male      No          0      Graduate              0             4166   
39     Male      No          0  Not Graduate              0             3748   
40     Male      No          0      Graduate              0             3600   
41     Male      No          0      Graduate              0             1800   
42     Male     Yes          0      Graduate              0             2400   
43     Male     Yes          0      Graduate              0             3941   
44     Male     Yes          0  Not Graduate              1             4695   
45   Female      No          0      Graduate              0             3410   
46     Male     Yes          1      Graduate              0             5649   
47     Male     Yes          0      Graduate              0             5821   
48   Female     Yes          0      Graduate              0             2645   
49   Female      No          0      Graduate              0             4000   
50   

In [353]:
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.ensemble import RandomForestClassifier
train = pd.read_csv('train_loan.csv')
test = pd.read_csv('test_loan.csv')
train['Loan_Status'] = train['Loan_Status'].apply(lambda x: 0 if x=='N' else 1 )
train['Self_Employed'].fillna((train['Self_Employed'].mode()[0]), inplace=True)
train['Self_Employed'] = train['Self_Employed'].apply(lambda x: 0 if x=='No' else 1 )
train['LoanAmount'].fillna((train['LoanAmount'].mean()), inplace=True)
train['Dependents'] =  train['Dependents'].replace('3+',3)
train['Dependents'].fillna((train['Dependents'].mode()[0]), inplace=True)
train['Credit_History'].fillna((train['Credit_History'].mode()[0]), inplace=True)
train['Gender'].fillna((train['Gender'].mode()[0]), inplace=True)
train['Gender'] = train['Gender'].astype("category")
train['Loan_Amount_Term'].fillna((train['Loan_Amount_Term'].mode()[0]), inplace=True)
train['Education'] = train['Education'].astype("category")
train['Property_Area'] = train['Property_Area'].astype("category")
train=train[((train.ApplicantIncome - train.ApplicantIncome.mean()) / train.ApplicantIncome.std()).abs() < 3]
train=train[((train.CoapplicantIncome - train.CoapplicantIncome.mean()) / train.CoapplicantIncome.std()).abs() < 3]
train['Total_Income'] = train['ApplicantIncome'] + train['CoapplicantIncome']
train['LoanAmtPerMon'] = train['LoanAmount'] / train['Loan_Amount_Term']
train['TotIncomeByLoan'] =  train['Total_Income'] /  train['LoanAmount']
new_train=train
new_train_loan_Status = new_train['Loan_Status']
new_train=new_train.drop(['Loan_Status'],axis=1)
new_train=new_train.drop(['Loan_ID'],axis=1)


#Test
test['Self_Employed'].fillna((test['Self_Employed'].mode()[0]), inplace=True)
test['Self_Employed'] = test['Self_Employed'].apply(lambda x: 0 if x=='No' else 1 )
test['LoanAmount'].fillna((test['LoanAmount'].mean()), inplace=True)
test['Dependents'] =  test['Dependents'].replace('3+',3)
test['Dependents'].fillna((test['Dependents'].mode()[0]), inplace=True)
test['Credit_History'].fillna((test['Credit_History'].mode()[0]), inplace=True)
test['Gender'].fillna((test['Gender'].mode()[0]), inplace=True)
test['Gender'] = test['Gender'].astype("category")
test['Loan_Amount_Term'].fillna((test['Loan_Amount_Term'].mode()[0]), inplace=True)
test['Education'] = test['Education'].astype("category")
test['Property_Area'] = test['Property_Area'].astype("category")
test['Total_Income'] = test['ApplicantIncome'] + test['CoapplicantIncome']
test['LoanAmtPerMon'] = test['LoanAmount'] / test['Loan_Amount_Term']
test['TotIncomeByLoan'] =  test['Total_Income'] /  test['LoanAmount']
loan_id_list=test['Loan_ID']
new_test=test.drop(['Loan_ID'],axis=1)
#Mark New Train with Training Column and Test with Test Column. If we use get_dummies on different data set
#Its possible that data set cannot match due to categorical values
new_train['Label'] = 1
new_test['Label'] = 0
# Concat
concat_df = pd.concat([new_train , new_test])
# Create your dummies
features_df = pd.get_dummies(concat_df)
# Split your data
train_df = features_df[features_df['Label'] == 1]
train_df=train_df.drop(['Label'],axis=1)
test_df = features_df[features_df['Label'] == 0]
test_df= test_df.drop(['Label'],axis=1)
#Running Random Forest
Random_forest = RandomForestClassifier(n_estimators=100, random_state = 42).fit(train_df,new_train_loan_Status)
pred_Random_forest = Random_forest.predict(test_df)
pred_Random_forest
final_test=test
final_test['Loan_Status']=pred_Random_forest
#Convert 1 0 to Y N
final_test.Loan_Status[final_test.Loan_Status == 1 ] = 'Y'
final_test.Loan_Status[final_test.Loan_Status == 0 ] = 'N'
final_test['Loan_ID']=loan_id_list
final_test[['Loan_ID','Loan_Status']].to_csv('AkFinal1.csv',index=False)


C:\Akshay\Train\Applications\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Akshay\Train\Applications\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
